# MODEL

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F
gpu = 'cuda' if torch.cuda.is_available() else 'cpu'
print(gpu)
# for reproducibility
torch.manual_seed(777)
if gpu == 'cuda':
    torch.cuda.manual_seed_all(777)


class CNNLSTM(nn.Module):

    def __init__(self,input_size,hidden_size,dropout,batch_size,bidirectional = False):
      super(CNNLSTM, self).__init__()
      self.conv1 = nn.Conv2d(1, 10, 3, padding=1)
      self.conv2 = nn.Conv2d(10,20,2, padding=1)
      self.conv3 = nn.Conv2d(20,1,2,padding=1)
      self.pool = nn.MaxPool2d(2, stride = 1)
      self.input_size = input_size
      self.rnn = nn.LSTM(input_size -1, (input_size -1)*5, num_layers = 3, dropout=dropout, batch_first=True, bidirectional=bidirectional)
      self.fc1 = nn.Linear(10,20)
      self.fc2 = nn.Linear(20,10)
      self.fc3 = nn.Linear(10,3)
      self.batch_size = batch_size

    def forward(self, input):
      output = input 
      convolution = self.conv1(output)
      pooling = self.pool(convolution)
      convolution = self.conv2(pooling)
      pooling = self.pool(convolution)
      convolution = self.conv3(pooling)
      pooling = self.pool(convolution)
      alstm = torch.zeros(self.batch_size,1,(self.input_size -1)*5).to(gpu)
      for i in range(pooling.shape[1]):
        poolingi = pooling[:,i,:,:]
        lstm , _= self.rnn(poolingi)
        lstm = lstm[:,-1,:]
        alstm[:,i,:] = lstm
      alstm = alstm.reshape(self.batch_size,10)
      fc = self.fc1(alstm)
      fc = self.fc2(fc)
      final = self.fc3(fc)
      weight = torch.zeros(final.shape[0],final.shape[1]).to(gpu)
      for j in range(final.shape[0]):
        a = final[j,0]
        b = final[j,1]
        c = final[j,2]
        weight[j,0] = a*10/(a+b+c)
        weight[j,1] = b*10/(a+b+c)
        weight[j,2] = c*10/(a+b+c)
      final = final.mul(weight)
      final = F.softmax(final,dim = -1)
      return final

# model = CNNLSTM(3,15,0.2,1000).to(gpu)
# a = torch.randn(1000,1,30,3).to(gpu)
# output = model(a)
# print(output.shape)
model = CNNLSTM(3,15,0.2,20000).to(gpu)
train_n = np.load('/content/drive/MyDrive/LSTM DATAre/train_lstm.npy')
volume_n = np.load('/content/drive/MyDrive/LSTM DATAre/volume_lstm.npy')
TF_n = np.load('/content/drive/MyDrive/LSTM DATAre/TFce1_lstm.npy')
traina = torch.FloatTensor(train_n)
train = traina[40000:60000,:,:].to(gpu)
train = train.reshape(20000,1,30,3).to(gpu)
trainv = traina[60000:,:,:].to(gpu)
volumea = torch.FloatTensor(volume_n)
volume = volumea[40000:60000,:].to(gpu)
volumev = volumea[60000:,:].to(gpu)
TFa = torch.LongTensor(TF_n)
TF = TFa[40000:60000].to(gpu)
print(TF.shape)
TFv = TFa[60000:].to(gpu)

optimizer = optim.Adam(model.parameters(), lr=0.01)
best_model_wts = model.state_dict() 
best_acc = 0
best_epoch = 0
l_array = []
a_array = []
criterion = nn.CrossEntropyLoss()
for epoch in range(250):
  for i in range(2):
    if i == 0:
      output = model(train)
      # loss = (torch.abs((output - TF))).mean()
      loss = criterion(output, TF)
      
      loss1 = loss.tolist()
      l_array.append(loss1)

      if (epoch + 1) % 2 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    else:
      pass
    
    # if i == 1:
    #   pred = model(trainv,volumev)
    #   accuracy = criterion(pred,TFv)
    #   ac1 = accuracy.tolist()
    #   a_array.append(ac1)
    #   if best_acc > accuracy:
    #     best_model_wts = model.state_dict()
    #     best_acc = accuracy
    #     best_ephoch = epoch
    #   if (epoch+1) % 10 == 0:
    #     print('Epoch:', '%04d' % (epoch + 1), 'accuracy =', '{:.6f}'.format(accuracy))
    # else:
    #   pass

plt.plot(l_array, 'r')
plt.plot(a_array,'b')
plt.show()
model_save_name = 'CNN_LSTMn.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

cuda
torch.Size([20000])
Epoch: 0002 cost = 1.406768
Epoch: 0004 cost = 1.390399
Epoch: 0006 cost = 1.390399
Epoch: 0008 cost = 1.390399
Epoch: 0010 cost = 1.390399


KeyboardInterrupt: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F

a = torch.randn(40,1,40,3)
print(a.shape)
conv1 = nn.Conv2d(1,10,3,padding = 1)
conv2 = nn.Conv2d(10,20,2,padding = 1)
conv3 = nn.Conv2d(20,10,2,padding = 1)
pool = nn.MaxPool2d(2, stride = 1)
print(pool)
output = conv1(a)
print(output.shape)
output = pool(output)
print(output.shape)
output = conv2(output)
print(output.shape)
output = pool(output)
print(output.shape)
output = conv3(output)
print(output.shape)
output = pool(output)
print(output.shape)

torch.Size([40, 1, 40, 3])
MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
torch.Size([40, 10, 40, 3])
torch.Size([40, 10, 39, 2])
torch.Size([40, 20, 40, 3])
torch.Size([40, 20, 39, 2])
torch.Size([40, 10, 40, 3])
torch.Size([40, 10, 39, 2])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F

a = torch.FloatTensor([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
a = a.reshape(2,6)
print(a)
b = torch.randn(6,2,3)
print(b)
print(b.reshape(6,6))

# 새 섹션

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F
gpu = 'cuda' if torch.cuda.is_available() else 'cpu'
print(gpu)
# for reproducibility
torch.manual_seed(777)
if gpu == 'cuda':
    torch.cuda.manual_seed_all(777)


class CNNLSTM(nn.Module):

    def __init__(self,input_size,hidden_size,dropout,batch_size,bidirectional = False):
      super(CNNLSTM, self).__init__()
      self.conv1 = nn.Conv2d(1, 20, 3, padding=1)
      self.conv2 = nn.Conv2d(20,40,2, padding=1)
      self.conv3 = nn.Conv2d(40,20,2,padding=1)
      self.pool = nn.MaxPool2d(2, stride = 1)
      self.input_size = input_size
      self.rnn = nn.LSTM(input_size -1, (input_size -1)*5, num_layers = 3, dropout=dropout, batch_first=True, bidirectional=bidirectional)
      self.fc1 = nn.Linear(200,100)
      self.fc2 = nn.Linear(100,50)
      self.fc3 = nn.Linear(50,3)
      self.batch_size = batch_size

    def forward(self, input):
      output = input 
      convolution = self.conv1(output)
      pooling = self.pool(convolution)
      convolution = self.conv2(pooling)
      pooling = self.pool(convolution)
      convolution = self.conv3(pooling)
      pooling = self.pool(convolution)
      alstm = torch.zeros(self.batch_size,20,(self.input_size -1)*5).to(gpu)
      for i in range(pooling.shape[1]):
        poolingi = pooling[:,i,:,:]
        lstm , _= self.rnn(poolingi)
        lstm = lstm[:,-1,:]
        alstm[:,i,:] = lstm
      for k in range(self.batch_size):
        alstm[k,]

      for j in range(final.shape[0]):
        a = final[j,0]
        b = final[j,1]
        c = final[j,2]
        weight[j,0] = a*10/(a+b+c)
        weight[j,1] = b*10/(a+b+c)
        weight[j,2] = c*10/(a+b+c)
      final = final.mul(weight)
      final = F.softmax(final,dim = -1)
      return final

# model = CNNLSTM(3,15,0.2,1000).to(gpu)
# a = torch.randn(1000,1,30,3).to(gpu)
# output = model(a)
# print(output.shape)
model = CNNLSTM(3,15,0.2,20000).to(gpu)
train_n = np.load('/content/drive/MyDrive/LSTM DATAre/train_lstm.npy')
volume_n = np.load('/content/drive/MyDrive/LSTM DATAre/volume_lstm.npy')
TF_n = np.load('/content/drive/MyDrive/LSTM DATAre/TFce1_lstm.npy')
traina = torch.FloatTensor(train_n)
train = traina[40000:60000,:,:].to(gpu)
train = train.reshape(20000,1,30,3).to(gpu)
trainv = traina[60000:,:,:].to(gpu)
volumea = torch.FloatTensor(volume_n)
volume = volumea[40000:60000,:].to(gpu)
volumev = volumea[60000:,:].to(gpu)
TFa = torch.LongTensor(TF_n)
TF = TFa[40000:60000].to(gpu)
print(TF.shape)
TFv = TFa[60000:].to(gpu)

optimizer = optim.Adam(model.parameters(), lr=0.01)
best_model_wts = model.state_dict() 
best_acc = 0
best_epoch = 0
l_array = []
a_array = []
criterion = nn.CrossEntropyLoss()
for epoch in range(250):
  for i in range(2):
    if i == 0:
      output = model(train)
      # loss = (torch.abs((output - TF))).mean()
      loss = criterion(output, TF)
      
      loss1 = loss.tolist()
      l_array.append(loss1)

      if (epoch + 1) % 2 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    else:
      pass
    
    # if i == 1:
    #   pred = model(trainv,volumev)
    #   accuracy = criterion(pred,TFv)
    #   ac1 = accuracy.tolist()
    #   a_array.append(ac1)
    #   if best_acc > accuracy:
    #     best_model_wts = model.state_dict()
    #     best_acc = accuracy
    #     best_ephoch = epoch
    #   if (epoch+1) % 10 == 0:
    #     print('Epoch:', '%04d' % (epoch + 1), 'accuracy =', '{:.6f}'.format(accuracy))
    # else:
    #   pass

plt.plot(l_array, 'r')
plt.plot(a_array,'b')
plt.show()
model_save_name = 'CNN_LSTMn.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

cuda


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


torch.Size([20000])
Epoch: 0002 cost = 0.792742
Epoch: 0004 cost = 0.715487
Epoch: 0006 cost = 0.712496
Epoch: 0008 cost = 0.712496
